Create the OpenShift Service definition below to open up the port 8080 of the notebook to network traffic in the namespace.

In [1]:
%%bash
echo "
kind: Service
apiVersion: v1
metadata:
  name: rshiny-devel
  namespace: $(cat /var/run/secrets/kubernetes.io/serviceaccount/namespace)
spec:
  ports:
    - protocol: TCP
      port: 8080
      targetPort: 8080
  type: ClusterIP
  selector:
    notebook-name: pecan-unconstrained-forecast
" | oc apply -f -
echo DONE

service/rshiny-devel created
DONE


Create the OpenShift NetworkPolicy definition below allow access from the route to the port 8080 of the notebook pod.

In [3]:
%%bash
echo "
kind: NetworkPolicy
apiVersion: networking.k8s.io/v1
metadata:
  name: rshiny-devel
spec:
  podSelector:
    matchLabels:
      notebook-name: pecan-unconstrained-forecast
  ingress:
    - ports:
        - protocol: TCP
          port: 8080
  policyTypes:
    - Ingress
status: {}
" | oc apply -f -

networkpolicy.networking.k8s.io/rshiny-devel created


Create the OpenShift Route definition below to expose the running RShiny application in development to the Internet.

In [5]:
%%bash
echo "
kind: Route
apiVersion: route.openshift.io/v1
metadata:
  name: rshiny-devel
  labels:
    notebook-name: pecan-unconstrained-forecast
spec:
  to:
    kind: Service
    name: rshiny-devel
    weight: 100
  port:
    targetPort: 8080
  tls:
    termination: edge
  wildcardPolicy: None
" | oc apply -f -

echo "https://$(oc get route/rshiny-devel -o jsonpath={.spec.host})"
echo DONE

route.route.openshift.io/rshiny-devel unchanged
https://rshiny-devel-pecan.apps-crc.testing
DONE
